# <a id='toc1_'></a>[_CLusters_: búsqueda, _plot_ y PCA](#toc0_)

----
**Table of contents**<a id='toc0_'></a>    

- [General](#toc1_1_)    
- [Funciones](#toc1_2_)    
  - [Para los promedios](#toc1_2_1_)    
  - [Torres a zonas](#toc1_2_2_)    
  - [Ploteo](#toc1_2_3_)    
- [_Main_](#toc1_3_)    
- [_Map_](#toc1_4_)    
  - [Zonas del cluster 0 (azul) vs zonas del cluster 1 (rojo)](#toc1_4_1_)    
   

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
----

## <a id='toc1_1_'></a>[General](#toc0_)

In [ ]:

import pickle
import os
os.environ['OMP_NUM_THREADS']='1'
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdmimport matplotlib.pyplot as plot
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA  

import json
import folium
from shapely.geometry import MultiPoint, Polygon, LineString
from shapely import geometry as shg


In [ ]:
general_directory=os.getcwd()+'/Data/'

cases=['allt','hw','no','mo']
st=['st','unst']

times=[i*3600 for i in range(0,25)]

times_r=[str(int(times[i]/3600))+'-'+str(int(times[i+1]/3600)) for i in range(len(times)-1)]

with open(general_directory+'Saves/Towers.pkl','r+b') as pk:
        towers=pickle.load(pk)
        
towers_prop=pd.read_csv(general_directory+'/zt_havana_tower.csv').drop_duplicates().drop_duplicates(subset=['id','category'],keep='first')

In [ ]:
for i in list(towers_prop['id'].unique()):
    if 99.5>=sum(towers_prop[towers_prop['id']==i]['percent']) or sum(towers_prop[towers_prop['id']==i]['percent'])>=100.5:
        towers_prop.drop(towers_prop[towers_prop['id']==i].index[-1],inplace=True)

## <a id='toc1_2_'></a>[Funciones](#toc0_)

In [ ]:
def save_dataP(name,dictionary,location=general_directory):

    with open(location+'Saves/'+ name+'.pkl', 'wb') as tf:
        pickle.dump(dictionary,tf)
    

### <a id='toc1_2_1_'></a>[Para los promedios](#toc0_)

In [ ]:
def load_hist(case,st,days):
    if st=='all': st='unst'
    elif st=='stable':st='st'
        
    with open(general_directory+'Saves/Meanh_'+case+'_'+st+'_'+days+'.pkl','r+b') as pk:
        hist=pickle.load(pk)
    return hist

In [ ]:
def load_mat(case,st,days):
    m=np.genfromtxt(general_directory+'Saves/Meanm_'+case+'_'+st+'_'+days+'.csv',delimiter=',')
    return m

### <a id='toc1_2_2_'></a>[Torres a zonas](#toc0_)

In [ ]:
def matrix_z(matrix_t):
    matrix_z=np.zeros((134,134))
    for i in range(len(towers)):
        for j in range(len(towers)):
            erow=towers_prop[towers_prop['id']==towers[i]]
            ecol=towers_prop[towers_prop['id']==towers[j]]
            for r in range(len(erow['category'])):
                for c in range(len(ecol['category'])):
                    matrix_z[int(erow['category'].iloc[r])-1][int(ecol['category'].iloc[c])-1]+=matrix_t[i][j]*erow['percent'].iloc[r]*ecol['percent'].iloc[c]/10000
    winsound.Beep(30000,500)
    return matrix_z
            
    
def hist_z(hist_t):
    hist_z=pd.DataFrame({i+' In':0 for i in times_r}|{i+' Out':0 for i in times_r},index=range(1,135))
    for i in hist_t.index:
        erow=towers_prop[towers_prop['id']==i]
        for r in range(len(erow['category'])):
                hist_z.loc[int(erow['category'].iloc[r])]+=hist_t.loc[i]*erow['percent'].iloc[r]/100
    return hist_z

### <a id='toc1_2_3_'></a>[Ploteo](#toc0_)

In [ ]:


def plots(hist,tow):
    plot.figure()  
    hist.iloc[tow][24:48].plot(kind='line',linestyle='--',label='Iniciados en la torre '+str(tow))
    hist.iloc[tow][:24].plot(kind='line',linestyle='--',label='Terminados en la torre '+str(tow))

    ax = plot.subplot()
    ax.set_xticks(range(24)) 
    ax.set_xticklabels(times_r, rotation=90) 

    plot.title('Data/Images/Inicios Vs finales en '+str(tow))
    plot.legend()
    plot.show()
    plot.savefig('Data/Images/Inicios Vs finales en '+str(tow) +'.jpg')


In [ ]:

def plots_T(hist,out_or_in,case,st,days):
    plot.figure() 

    for t in range(len(hist)):
        if out_or_in=='Inicios': hist.loc[t][:24].plot(kind='line',linestyle='--',label='Cluster '+str(t))
        elif out_or_in=='Finales':hist.loc[t][24:48].plot(kind='line',linestyle='--',label='Cluster '+str(t))
        ax = plot.subplot()
        ax.set_xticks(range(24)) 
        ax.set_xticklabels(times_r, rotation=90) 
        plot.legend()
        plot.title(out_or_in+' ('+case+' travels,'+st+' citizens,'+days+'days)')

    plot.savefig('Data/Images/Clusters '+out_or_in+' ('+case+' travels,'+st+' citizens,'+days+'days).jpg')
    

----
## <a id='toc1_3_'></a>[_Main_](#toc0_)

In [ ]:
def main(cases,st,days):

    nclus=2
    
    with tqdm(desc="Clusterizing and plotting",total=4*(len(cases)*len(st)*len(days))) as pbar:
        for cs in cases:
            for s in st:
                for day in days:
                    %matplotlib inline
                    datos  = hist_z(load_hist(cs,s,day))
                    datos=datos.loc[~(datos==0).all(axis=1)]
                    datos_norm = datos.div(datos.sum(axis=1), axis=0).fillna(0)
                    
                    
                    wcss = []

                    for i in range (1,11):
                        kmeans  = KMeans (n_clusters = i,max_iter = 300)
                        kmeans.fit (datos_norm)
                        wcss.append (kmeans.inertia_)


                    plot.plot (range(1,11),wcss)
                    plot.xlabel ('Número de clusters')
                    plot.ylabel ('wcss')
                    plot.title ('Codo de Jambú ('+cs+' travels,'+s+' citizens,'+day+'days)')
                    plot.savefig('Data/Images/Codo de Jambú ('+cs+' travels,'+s+' citizens,'+day+'days).jpg')
                    pbar.update()
                    
                    clustering = KMeans (n_clusters = nclus,max_iter = 300)
                    clustering.fit (datos_norm)

                    datos['KMeans_clustering'] = clustering.labels_

                    pca = PCA(n_components = 2)
                    pca_datos = pca.fit_transform (datos_norm)
                    pca_datos_df = pd.DataFrame (data = pca_datos, columns = ['Componente_1','Componente_2'] )
                    pca_nombres_datos = pd.concat ([pca_datos_df,datos['KMeans_clustering']],axis=1)

                    fig = plot.figure (figsize = (6,6))  
                    ax = fig.add_subplot (1,1,1)
                    ax.set_xlabel ('Componente 1')
                    ax.set_ylabel ('Componente 2')
                    plot.title ('Clusters scatter ('+cs+' travels,'+s+' citizens,'+day+'days)')
                    ax.scatter (x = pca_nombres_datos.Componente_1,y=pca_nombres_datos.Componente_2,c=pca_nombres_datos.KMeans_clustering,s=50,cmap='viridis')
                    plot.savefig('Data/Images/Clusters scatter ('+cs+' travels,'+s+' citizens,'+day+'days).jpg')
                    pbar.update()
                    
                    datos_norm=pd.concat ([datos_norm,datos['KMeans_clustering']],axis=1).dropna()
            
                    c_h=pd.DataFrame([datos_norm[datos_norm.KMeans_clustering==i].mean() for i in range (nclus)], index=list(range (nclus)))

                    plots_T(c_h,'Inicios',cs,s,day)
                    pbar.update()
                    plots_T(c_h,'Finales',cs,s,day)
                    pbar.update()
                    
                    print('cluster 1 :',list(datos[datos.KMeans_clustering==0].index-1))
                    print('cluster 2 :',list(datos[datos.KMeans_clustering==1].index-1))

In [ ]:
cluster 1 : [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 29, 30, 31, 32, 33, 34, 36, 37, 39, 40, 41, 46, 54, 55, 56, 57, 58, 59, 66, 67, 68, 69, 73, 80, 90, 92, 96, 102, 107, 108, 109, 111, 114, 116, 117, 118, 121, 127, 133]
cluster 2 : [0, 7, 38, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 60, 61, 62, 63, 64, 65, 70, 71, 72, 74, 75, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 88, 89, 91, 93, 94, 95, 97, 98, 99, 100, 101, 103, 104, 105, 106, 110, 112, 113, 115, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132]

cluster 1 : [0, 1, 3, 7, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 61, 62, 63, 64, 65, 66, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 110, 112, 113, 115, 118, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132]
cluster 2 : [2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 46, 54, 55, 56, 57, 58, 59, 60, 67, 68, 69, 73, 108, 109, 111, 114, 116, 117, 121, 127, 133]


In [ ]:
main(['allt'],['stable','all'],['week','all'])

In [ ]:
nclus=2
datos  = hist_z(load_hist('allt','all','all'))
datos=datos.loc[~(datos==0).all(axis=1)]
datos_norm = datos.div(datos.sum(axis=1), axis=0).fillna(0)
clustering = KMeans (n_clusters = nclus,max_iter = 300)
clustering.fit (datos_norm)

datos['KMeans_clustering'] = clustering.labels_



In [ ]:
l1= [0, 7, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 61, 62, 63, 64, 65, 66, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 110, 112,113, 115, 119, 120, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132]
l2=[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 46, 54, 55, 56, 57, 58, 59, 60, 67, 68, 69, 73, 107, 108, 109, 111, 114, 116, 117, 118, 121, 127,133]



In [ ]:
for i in l2: print(i in list(datos[datos.KMeans_clustering==1].index-1))

----
## <a id='toc1_4_'></a>[_Map_](#toc0_)

In [ ]:
def convert_geo_to_shg(features):
    features_shg = []
    shg_props = []
    for i in range(len(features)):
        feature =features[i]
        #print(" working with ",i," ",feature['properties']," \n ============ \n\n ")
        shape = shg.shape(feature['geometry'])
        features_shg.append(shape)
        shg_props.append(feature['properties'])
    return features_shg, shg_props

In [ ]:
fd = open('./zonas_de_trasporte.json', 'r')
transp_zones_json = json.load(fd)
fd.close()

transp_zones_shg, transp_props = convert_geo_to_shg(transp_zones_json['features'])

In [ ]:
zones0=list((set(range(134))-set(datos[datos.KMeans_clustering==1].index-1))-set(datos[datos.KMeans_clustering==0].index-1))

In [ ]:
transp_zones_json0={'type': 'FeatureCollection',
 'name': 'zonas_de_trasporte',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features':[transp_zones_json['features'][z] for z in zones0]}

transp_zones_json1={'type': 'FeatureCollection',
 'name': 'zonas_de_trasporte',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features':[transp_zones_json['features'][z] for z in list(datos[datos.KMeans_clustering==0].index-1)]}


transp_zones_json2={'type': 'FeatureCollection',
 'name': 'zonas_de_trasporte',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features':[transp_zones_json['features'][z] for z in list(datos[datos.KMeans_clustering==1].index-1)]}


### <a id='toc1_4_1_'></a>[Zonas del cluster 0 (azul) vs zonas del cluster 1 (rojo)](#toc0_)

In [ ]:
map = folium.Map(location=[23.0826, -82.2845], zoom_start=11, tiles='openstreetmap')

zones_geojson1 = folium.GeoJson(transp_zones_json1, 
                               style_function = lambda x: {'weight': 1.5, 'fillOpacity' : .1,'color' : 'blue'}, 
                               tooltip=folium.features.GeoJsonTooltip(fields = ['NO_DE_ZONA']))

zones_geojson2 = folium.GeoJson(transp_zones_json2, 
                               style_function = lambda x: {'weight': 1.5, 'fillOpacity' : .1,'color' : 'red'}, 
                               tooltip=folium.features.GeoJsonTooltip(fields = ['NO_DE_ZONA']))

zones_geojson0 = folium.GeoJson(transp_zones_json0, 
                               style_function = lambda x: {'weight': 1.5, 'fillOpacity' : .1,'color' : 'black'}, 
                               tooltip=folium.features.GeoJsonTooltip(fields = ['NO_DE_ZONA']))
# Para dejar un solo layer cometar aqui

zones_geojson0.add_to(map)
zones_geojson1.add_to(map)
zones_geojson2.add_to(map)

In [ ]:
map
